In [51]:
import numpy as np
import MDAnalysis as mda
import mdtraj as md

from funcs_characterise import *
from paths import *

from pathlib import Path
import os

In [48]:
dfg_in_samples = md.load("./data_egfr/msm/dunbrack_studies/lag100_random_trials/61/MS_0.pdb")
dfg_out_samples = md.load("./data_egfr/msm/dunbrack_studies/lag100_random_trials/61/MS_1.pdb")

In [49]:
dfg_in_mean, dfg_in_std = cal_within_state_rmsd(dfg_in_samples)
dfg_out_mean, dfg_out_std = cal_within_state_rmsd(dfg_out_samples)
in_out_mean, in_out_std = cal_between_states_rmsd(dfg_in_samples, dfg_out_samples)
score = cal_between_states_diff(in_out_mean, dfg_in_std, dfg_out_std)

In [52]:
save_example_with_property("./data_egfr/msm/dunbrack_studies/lag100_random_trials/61/MS_0.pdb", score, 'rmsd_diff_dfg-in-out.pdb')

ValueError: 'H5' isn't a valid topology format, nor a coordinate format
   from which a topology can be minimally inferred.
   You can use 'Universe(topology, ..., topology_format=FORMAT)'
   to explicitly specify the format and
   override automatic detection. Known FORMATs are:
   dict_keys(['PSF', 'TOP', 'PRMTOP', 'PARM7', 'PDB', 'ENT', 'XPDB', 'PQR', 'GRO', 'CRD', 'PDBQT', 'DMS', 'TPR', 'MOL2', 'DATA', 'LAMMPSDUMP', 'XYZ', 'TXYZ', 'ARC', 'GMS', 'CONFIG', 'HISTORY', 'XML', 'MMTF', 'GSD', 'MINIMAL', 'ITP', 'IN', 'FHIAIMS', 'PARMED', 'RDKIT', 'OPENMMTOPOLOGY', 'OPENMMAPP'])
   See https://docs.mdanalysis.org/documentation_pages/topology/init.html#supported-topology-formats
   For missing formats, raise an issue at 
   https://github.com/MDAnalysis/mdanalysis/issues